In [1]:
% cd ~/Documents/GitHub/cs231n-adni/new_code

/Users/elissali/Documents/GitHub/cs231n-adni/new_code


In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns

import os
import utils
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
import multiprocessing
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [3]:
import data, model, interpretation, utils, vis_utils

torch.Size([32, 1])


In [4]:
train_data = np.load("train_aug.npy")
val_data = np.load("val_aug.npy")
test_data = np.load("test_aug.npy")
train_label = np.load("train_label_aug.npy")
val_label = np.load("val_label_aug.npy")
test_label = np.load("test_label_aug.npy")

train_data = np.reshape(train_data, (2048, 1, 64, 64, 64))
val_data = np.reshape(val_data, (512, 1, 64, 64, 64))
test_data = np.reshape(test_data, (512, 1, 64, 64, 64))

# Visualisation

In [13]:
train_data[0].shape

(1, 64, 64, 64)

In [14]:
train_data = np.reshape(train_data, (2048, 1, 64, 64, 64))      # want channels first!
test_data = np.reshape(test_data, (512, 1, 64, 64, 64))

In [25]:
neg_ex = torch.squeeze(torch.from_numpy(test_data[11]))
pos_ex = torch.squeeze(torch.from_numpy(test_data[500]))

print(pos_ex.shape)
# vis_utils.plot_slices(neg_ex)

torch.Size([64, 64, 64])


In [26]:
net = model.SingleTimestep3DCNN(in_num_ch=1, img_size=(64,64,64), inter_num_ch=16, fc_num_ch=16,
                                conv_act='relu', fc_act='tanh').to(torch.device('cpu'))

# net.load_state_dict(torch.load('../ckpt/2020_5_5_21_44/model_best.pth.tar')['model'])
net.load_state_dict(torch.load('../ckpt/2020_5_18_16_42/epoch031.pth.tar')['model'])

<All keys matched successfully>

In [20]:
net.eval();

In [21]:
train_data[2000][0].shape
train_data[0][0].shape

(64, 64, 64)

# Confounder

In [64]:
ex = torch.Tensor([test_data[0]])
ex.shape

torch.Size([1, 1, 64, 64, 64])

In [77]:
features = net.feature_extractor(ex)
print(features.shape)
flat_features = features.view(ex.shape[0], -1)
print(flat_features.shape)

torch.Size([1, 16, 4, 4, 4])
torch.Size([1, 1024])


In [74]:
net.fc3

Linear(in_features=32, out_features=1, bias=True)

https://discuss.pytorch.org/t/insert-new-layer-in-the-middle-of-a-pre-trained-model/12414/5

In [76]:
class Confounder3DCNN(nn.Module):
    def __init__(self, in_num_ch=1, img_size=(32,64,64), inter_num_ch=16, fc_num_ch=16,
                conv_act='relu', fc_act='tanh'):
        super(Confounder3DCNN, self).__init__()

        self.feature_extractor = net.feature_extractor # use feature extractor from existing model
        num_feat = int(inter_num_ch * (img_size[0]*img_size[1]*img_size[2]) / ((2**4)**3))

        num_output = 1
        self.num_cls = 2

        self.fc1 = net.fc1    # use fc1 from existing model
        self.fc2 = net.fc1    # use fc2 from existing model
        self.fc3 = net.fc3    # use fc3 from existing model

        self.init_model()

    def init_model(self):
        for layer in [self.fc1, self.fc2, self.fc3]:
            for name, weight in layer.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_normal_(weight)
                if 'bias' in name:
                    nn.init.constant_(weight, 0.0)

    def forward(self, x):
        conv4 = self.feature_extractor(x)
        conv4_flatten = conv4.view(x.shape[0], -1)
        conv4_mask = BinaryMask(conv4_flatten, mask, pre_feature)   # apply binary_mask after conv4_flatten
        fc1 = self.fc1(conv4_mask)
        fc2 = self.fc2(fc1)
        output = self.fc3(fc2)
        return output

In [ ]:
# define a dummy layer
class BinaryMask(Layer):

    def __init__(self, output_dim, mask, pre_feature, **kwargs):
        self.output_dim = output_dim
        self.mask = mask
        self.pre_feature = pre_feature
        
        super(BinaryMask, self).__init__(**kwargs)

    def build(self, input_shape):
        self.mask_tensor = K.variable(self.mask)
        self.pre_feature = K.variable(self.pre_feature)
        
        super(BinaryMask, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return x * self.mask_tensor + self.pre_feature

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

I think what's happening here is:
- BinaryMask takes the original layer, then masks it, then adds back the original values (of the stuff it zeroed out?)
- The idea being that you "fix" the value of the confounded features/don't backprop them